In [6]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2

image_size = (480, 640)
batch_size = 32


train_dir = "/content/drive/My Drive/Colab Notebooks/new_chess_data/train_data"
test_dir = "/content/drive/My Drive/Colab Notebooks/new_chess_data/test_data"
valid_dir = "/content/drive/My Drive/Colab Notebooks/new_chess_data/validation_data"


num_classes = 12

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(480, 640),
    batch_size=batch_size,
    class_mode='categorical')


validation_generator = validation_datagen.flow_from_directory(
    valid_dir,
    target_size=(480,640),
    batch_size=batch_size,
    class_mode='categorical')


test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(480,640),
    batch_size=batch_size,
    class_mode='categorical')


model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.001),input_shape=(480,640,3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu',kernel_regularizer=l2(0.001)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (2, 2), activation='relu', kernel_regularizer=l2(0.001)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.001)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.6),
    tf.keras.layers.Dense(num_classes, activation='softmax')
    ])


Found 1017 images belonging to 12 classes.
Found 1002 images belonging to 12 classes.
Found 4331 images belonging to 12 classes.


In [39]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [40]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_47 (Conv2D)          (None, 478, 638, 32)      896       
                                                                 
 max_pooling2d_47 (MaxPoolin  (None, 239, 319, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_48 (Conv2D)          (None, 237, 317, 64)      18496     
                                                                 
 max_pooling2d_48 (MaxPoolin  (None, 118, 158, 64)     0         
 g2D)                                                            
                                                                 
 conv2d_49 (Conv2D)          (None, 117, 157, 128)     32896     
                                                                 
 max_pooling2d_49 (MaxPoolin  (None, 58, 78, 128)    

In [41]:
from keras.callbacks import EarlyStopping

earlystop_callback = EarlyStopping(
    monitor='val_accuracy', 
    min_delta=0.001, 
    patience=3, 
    verbose=1, 
    mode='auto',
    restore_best_weights=True
)

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[earlystop_callback]
)



Epoch 1/30
31/31 [==============================] - 42s 1s/step - loss: 2.8400 - accuracy: 0.0975 - val_loss: 2.5967 - val_accuracy: 0.1351
Epoch 2/30
31/31 [==============================] - 33s 1s/step - loss: 2.5484 - accuracy: 0.1472 - val_loss: 2.4736 - val_accuracy: 0.1956
Epoch 3/30
31/31 [==============================] - 33s 1s/step - loss: 2.3742 - accuracy: 0.2081 - val_loss: 2.1684 - val_accuracy: 0.2560
Epoch 4/30
31/31 [==============================] - 34s 1s/step - loss: 2.1749 - accuracy: 0.2853 - val_loss: 2.0082 - val_accuracy: 0.4083
Epoch 5/30
31/31 [==============================] - 33s 1s/step - loss: 1.9178 - accuracy: 0.3898 - val_loss: 1.7697 - val_accuracy: 0.4859
Epoch 6/30
31/31 [==============================] - 33s 1s/step - loss: 1.7172 - accuracy: 0.4203 - val_loss: 1.4886 - val_accuracy: 0.5000
Epoch 7/30
31/31 [==============================] - 34s 1s/step - loss: 1.5203 - accuracy: 0.4883 - val_loss: 1.3245 - val_accuracy: 0.5605
Epoch 8/30
31/31 [==